IMDb 電影資料庫
電影評論數據集   
50000篇文字評論並且被標記為正或是負
我們試著用將影評訓練集和標籤來建立模型並利用影評測試集計算正評和負評的正確率

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
由上列網址下載數據集  並解壓縮到與 Sentiment Analysis.ipynb 同一個資料夾


In [1]:
! pip install pyprind

  Running setup.py bdist_wheel for pyprind: started
  Running setup.py bdist_wheel for pyprind: finished with status 'done'
  Stored in directory: C:\Users\BIG DATA\AppData\Local\pip\Cache\wheels\cd\7b\7e\253b8df2dd2d8ea70d76870390300743f58a26317abb11e125
Successfully built pyprind


In [6]:
import pyprind
import pandas as pd
import os

# unzipped movie dataset

basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
# 內有五萬筆資料
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
# 將'test', 'train' 資料 'pos', 'neg' 兩個類別都寫入 df
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:27


In [10]:
pbar = pyprind.ProgBar(50000)
for idx in range(50000):
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [7]:
import pandas as pd
df.head(5)
# 資料還是有序的

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [8]:
import numpy as np
# 將資料順序打亂
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [9]:
df.head(10)

,review,sentiment
11841,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
19602,OK... so... I really like Kris Kristofferson a...,0
45519,"***SPOILER*** Do not read this, if you think a...",0
25747,hi for all the people who have seen this wonde...,1
42642,"I recently bought the DVD, forgetting just how...",0
31902,Leave it to Braik to put on a good show. Final...,1
30346,Nathan Detroit (Frank Sinatra) is the manager ...,1
12363,"To understand ""Crash Course"" in the right cont...",1
32490,I've been impressed with Chavez's stance again...,1
26128,This movie is directed by Renny Harlin the fin...,1


In [7]:
df.to_csv('./movie_data.csv', index=False, encoding='utf-8')
# 轉成CSV格式儲存

In [8]:
# 到這邊算是整理好資料的格式了  之後從這邊讀資料就好了
import pandas as pd

pd.read_csv('./movie_data.csv', encoding='utf-8')
df.head(10)

,review,sentiment
11841,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
19602,OK... so... I really like Kris Kristofferson a...,0
45519,"***SPOILER*** Do not read this, if you think a...",0
25747,hi for all the people who have seen this wonde...,1
42642,"I recently bought the DVD, forgetting just how...",0
31902,Leave it to Braik to put on a good show. Final...,1
30346,Nathan Detroit (Frank Sinatra) is the manager ...,1
12363,"To understand ""Crash Course"" in the right cont...",1
32490,I've been impressed with Chavez's stance again...,1
26128,This movie is directed by Renny Harlin the fin...,1


# 詞袋模型(bag-of-words model)

In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [10]:
print(count.vocabulary_)


{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [11]:
print(bag.toarray())
#{‘and’  ,’is’ ,'shining‘ , 'sun‘ , 'sweet‘ ,'the‘ , 'weather'}
# 向量中的值/各分量和, 也就是 Term Frequency (tf);  tf(t,d)

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


#  term frequency-inverse document frequency(tf-idf)


In [12]:
np.set_printoptions(precision=2)


In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())
# TfidfTransformer 轉換器,  從 CountVectorizer輸入原始詞頻, 轉成tf-idf

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


In [14]:
#用 "is" 比喻在第三個 document中 
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


In [15]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf

array([ 3.39,  3.  ,  3.39,  1.29,  1.29,  1.29,  2.  ,  1.69,  1.29])

In [16]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([ 0.5 ,  0.45,  0.5 ,  0.19,  0.19,  0.19,  0.3 ,  0.25,  0.19])

# Cleaning text data


In [22]:
df.loc[0, 'review'][-50:]


'and I suggest that you go see it before you judge.'

In [23]:
# 清除一些html標籤和表情符號
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [24]:
preprocessor(df.loc[0, 'review'][-50:])


'and i suggest that you go see it before you judge '

In [25]:
preprocessor("</a>This :) is :( a test :-)!")


'this is a test :) :( :)'

In [26]:
df['review'] = df['review'].apply(preprocessor)


# Processing documents into tokens (中文則需要斷詞)


In [20]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

# 分割成單獨的字詞
def tokenizer(text):
    return text.split()

#　還原成原型
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [27]:
tokenizer('runners like running and thus they run')


['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [24]:
tokenizer_porter('runners like running and thus they run')


['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [16]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\BIG
[nltk_data]     DATA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [28]:
from nltk.corpus import stopwords
# 停用字刪除 and, a...
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 用邏輯斯迴歸做分類

In [13]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
# 5 fold & grid-search 找最佳參數
# 可能要跑半個小時, 我把最佳參數挑出來  其他的先關掉了 
# n_jobs=1, 可能比較不會影響你們電腦其他東西運作

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
#               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer],

#               'clf__penalty': ['l1', 'l2'],
               'clf__penalty': ['l2'],
#               'clf__C': [1.0, 10.0, 100.0]},
                'clf__C': [10.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
#               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__tokenizer': [tokenizer],
               'vect__use_idf':[False],
               'vect__norm':[None],
#               'clf__penalty': ['l1', 'l2'],
               'clf__penalty': ['l2'],
#               'clf__C': [1.0, 10.0, 100.0]},
                'clf__C': [10.0]}
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1)

In [30]:
gs_lr_tfidf.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', '...199488>], 'vect__use_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l2'], 'clf__C': [10.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
     

In [26]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001B40DEC8620>} 
CV Accuracy: 0.892


In [27]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.897
